# WASM PTY Error Debugging Guide

This notebook documents the implementation and debugging of the enhanced PTY functionality in the WASM kernel, focusing on the terminal emulation, shell interface, and command execution features.

## Overview

The WASM kernel has been enhanced with:
- **Terminal Emulation**: VT100/ANSI escape sequence support with an 80x24 virtual screen
- **Shell Interface**: Built-in commands (help, ls, cd, pwd, ps, env, etc.) with environment variables and command history
- **PTY (Pseudo-Terminal)**: Enhanced PTY with canonical/raw modes, echo control, and line buffering
- **Command Execution**: Real command parsing and execution with output capture

## Current Issues

We're experiencing errors in the browser:
- `state.module.UTF8ToString is not a function`
- `state.module._malloc is not a function`

This indicates that the WASM runtime methods are not being properly exported or the module initialization is incomplete.

## 1. Reproducing the PTY Errors in the WASM Terminal

Let's examine the errors that occur when running the WASM Terminal component. These errors appear in the browser console when the PTY functionality tries to access WASM module functions.

### Error Console Output

```
PTY get screen error: TypeError: state.module.UTF8ToString is not a function
    at Object.ptyGetScreen (WasmKernelProvider.tsx:428:31)

PTY read error: TypeError: state.module._malloc is not a function
    at Object.ptyRead (WasmKernelProvider.tsx:298:39)
```

### Root Cause Analysis

These errors occur because:
1. The WASM module doesn't export the expected runtime methods
2. The exported functions list in the build configuration is incomplete
3. The module initialization timing might be incorrect

In [ ]:
// Terminal Component Initialization - React TypeScript
// File: src/plugins/apps/wasm-kernel/components/Terminal.tsx

const Terminal = ({ className }) => {
  const { api, state } = useWasmKernel();
  const [isConnected, setIsConnected] = useState(false);

  // This useEffect triggers the errors when WASM kernel initializes
  useEffect(() => {
    if (!api || !state.isInitialized) return;

    const initTerminal = async () => {
      try {
        // ERROR 1: This call fails with "UTF8ToString is not a function"
        const screen = await api.ptyGetScreen();
        setOutput(screen);
        setIsConnected(true);
      } catch (error) {
        console.error('Failed to initialize terminal:', error);
      }
    };

    initTerminal();
  }, [api, state.isInitialized]);

  // This polling also triggers errors
  useEffect(() => {
    if (!api || !isConnected) return;

    const pollOutput = async () => {
      try {
        // ERROR 2: This call fails with "_malloc is not a function"
        const newOutput = await api.ptyRead();
        if (newOutput) {
          setOutput(prev => prev + newOutput);
        }
      } catch (error) {
        console.error('Error polling PTY output:', error);
      }
    };

    const interval = setInterval(pollOutput, 100);
    return () => clearInterval(interval);
  }, [api, isConnected]);
};

## 2. Inspecting the useWasmKernel Hook and Kernel State

The `useWasmKernel` hook manages the WASM module lifecycle and provides the API interface. Let's examine its implementation and the structure of the returned objects.

In [ ]:
// File: src/hooks/useWasmKernel.ts
import { useContext } from 'react';
import { WasmKernelContext } from '../lib/wasm/WasmKernelProvider';

export const useWasmKernel = () => {
  const context = useContext(WasmKernelContext);
  
  if (!context) {
    throw new Error('useWasmKernel must be used within a WasmKernelProvider');
  }
  
  return context; // Returns { state, api, initialize }
};

// The actual module loading happens in WasmKernelProvider.tsx
// Key initialization sequence:

const wasmModule = await WasmCore({
  onRuntimeInitialized: function () {
    console.log("🚀 WASM Kernel runtime initialized");
    
    // This is where we should verify function availability
    if (!this.FS) {
      console.error("❌ FS object not available");
      return;
    }
    
    console.log("✅ FS API confirmed available:", typeof this.FS);
    
    // ISSUE: We're not checking for UTF8ToString, _malloc, etc.
    // before marking as initialized
    
    (this as EmscriptenModule).callMain();
    
    setState({
      isInitialized: true,
      isLoading: false,
      module: this as EmscriptenModule,
    });
  }
});

## 3. Debugging the PTY API Methods

Let's examine the problematic PTY API methods that are causing the errors and add diagnostic logging to understand what's happening.

In [ ]:
// File: src/lib/wasm/WasmKernelProvider.tsx
// Current problematic implementation:

ptyGetScreen: async (): Promise<string> => {
  if (!state.module) {
    throw new Error("WASM module not available");
  }
  
  try {
    // This line fails: state.module.UTF8ToString is not a function
    const ptr = state.module.ccall('wasm_pty_get_screen', 'number', [], []) as number;
    if (ptr === 0) {
      return "";
    }
    // ERROR HERE: UTF8ToString is not available
    return state.module.UTF8ToString(ptr);
  } catch (error) {
    console.error("PTY get screen error:", error);
    return "";
  }
},

ptyRead: async (): Promise<string> => {
  if (!state.module) {
    throw new Error("WASM module not available");
  }
  
  try {
    const hasData = state.module.ccall('wasm_pty_has_output', 'number', [], []);
    if (!hasData) return "";
    
    const bufferSize = 1024;
    // ERROR HERE: _malloc is not available
    const buffer = state.module._malloc(bufferSize);
    
    try {
      const bytesRead = state.module.ccall(
        'wasm_pty_read_output',
        'number',
        ['number', 'number'],
        [buffer, bufferSize]
      );
      
      if (bytesRead > 0) {
        // This would also fail with UTF8ToString
        const result = state.module.UTF8ToString(buffer, bytesRead);
        return result;
      }
      return "";
    } finally {
      state.module._free(buffer);
    }
  } catch (error) {
    console.error("PTY read error:", error);
    return "";
  }
};

## 4. Testing Module Initialization and Function Availability

Let's create diagnostic code to test if the required WASM module functions are actually available after initialization.

In [ ]:
// Diagnostic function to check WASM module completeness
function validateWasmModule(module: EmscriptenModule): {
  isValid: boolean;
  missing: string[];
  available: string[];
} {
  const requiredFunctions = [
    'FS',
    'callMain', 
    'ccall',
    'cwrap',
    'UTF8ToString',
    'stringToUTF8',
    '_malloc',
    '_free'
  ];
  
  const missing: string[] = [];
  const available: string[] = [];
  
  for (const funcName of requiredFunctions) {
    if (funcName in module && typeof module[funcName] === 'function') {
      available.push(funcName);
    } else {
      missing.push(funcName);
    }
  }
  
  return {
    isValid: missing.length === 0,
    missing,
    available
  };
}

// Enhanced onRuntimeInitialized with validation
const wasmModule = await WasmCore({
  onRuntimeInitialized: function () {
    console.log("🚀 WASM Kernel runtime initialized");
    
    // Validate the module before marking as ready
    const validation = validateWasmModule(this as EmscriptenModule);
    
    console.log("📊 WASM Module Validation:");
    console.log("  Available functions:", validation.available);
    console.log("  Missing functions:", validation.missing);
    console.log("  Is valid:", validation.isValid);
    
    if (!validation.isValid) {
      console.error("❌ WASM module is incomplete, missing:", validation.missing);
      setState({
        isLoading: false,
        error: `WASM module missing functions: ${validation.missing.join(', ')}`
      });
      return;
    }
    
    // Only proceed if validation passes
    try {
      (this as EmscriptenModule).callMain();
      console.log("✅ WASM Kernel main loop started");
      
      setState({
        isInitialized: true,
        isLoading: false,
        module: this as EmscriptenModule,
      });
    } catch (error) {
      console.error("❌ Failed to start kernel main loop:", error);
      setState({
        isLoading: false,
        error: error instanceof Error ? error.message : "Failed to start kernel"
      });
    }
  }
});

## 5. Adding Error Handling and Logging

Let's create robust error handling and logging for the PTY API methods to provide clearer diagnostics when functions are missing.

In [ ]:
// Improved PTY methods with comprehensive error handling

ptyGetScreen: async (): Promise<string> => {
  if (!state.module) {
    throw new Error("WASM module not available");
  }
  
  // Check for required functions before using them
  if (typeof state.module.ccall !== 'function') {
    throw new Error("WASM module missing ccall function");
  }
  
  if (typeof state.module.UTF8ToString !== 'function') {
    console.error("Available module properties:", Object.keys(state.module));
    throw new Error("WASM module missing UTF8ToString function - check build configuration");
  }
  
  try {
    const ptr = state.module.ccall('wasm_pty_get_screen', 'number', [], []) as number;
    if (ptr === 0) {
      console.warn("PTY get screen returned null pointer");
      return "";
    }
    
    const result = state.module.UTF8ToString(ptr);
    console.log(`PTY screen retrieved: ${result.length} characters`);
    return result;
  } catch (error) {
    console.error("PTY get screen error:", error);
    console.error("Module state:", {
      moduleExists: !!state.module,
      ccallExists: typeof state.module.ccall,
      utf8ToString: typeof state.module.UTF8ToString
    });
    return "";
  }
},

ptyRead: async (): Promise<string> => {
  if (!state.module) {
    throw new Error("WASM module not available");
  }
  
  // Validate all required functions upfront
  const requiredFunctions = ['ccall', '_malloc', '_free', 'UTF8ToString'];
  for (const funcName of requiredFunctions) {
    if (typeof state.module[funcName] !== 'function') {
      console.error(`Missing WASM function: ${funcName}`);
      console.error("Available functions:", Object.keys(state.module).filter(k => typeof state.module[k] === 'function'));
      throw new Error(`WASM module missing ${funcName} function - check build exports`);
    }
  }
  
  try {
    const hasData = state.module.ccall('wasm_pty_has_output', 'number', [], []);
    if (!hasData) {
      return "";
    }
    
    const bufferSize = 1024;
    const buffer = state.module._malloc(bufferSize);
    
    if (!buffer) {
      throw new Error("Failed to allocate WASM memory buffer");
    }
    
    try {
      const bytesRead = state.module.ccall(
        'wasm_pty_read_output',
        'number',
        ['number', 'number'],
        [buffer, bufferSize]
      );
      
      if (bytesRead > 0) {
        const result = state.module.UTF8ToString(buffer, bytesRead);
        console.log(`PTY read ${bytesRead} bytes: "${result}"`);
        return result;
      }
      
      return "";
    } finally {
      state.module._free(buffer);
    }
  } catch (error) {
    console.error("PTY read error:", error);
    console.error("Module capabilities:", {
      hasModule: !!state.module,
      ccall: typeof state.module?.ccall,
      malloc: typeof state.module?._malloc,
      free: typeof state.module?._free,
      utf8ToString: typeof state.module?.UTF8ToString
    });
    return "";
  }
};

# Terminal Behavior and PTY Mode Testing

## How PTY Terminal Modes Should Work

The terminal component supports four different PTY modes, each with distinct behavior:

### 1. RAW Mode
- **Input**: Each keystroke sent immediately to PTY
- **Echo**: No local echo, PTY controls all output
- **Use case**: Full-screen applications, games, editors like vi
- **UI**: No input field, captures keystrokes directly

### 2. ECHO Mode
- **Input**: Each keystroke sent immediately to PTY
- **Echo**: PTY echoes characters back to display
- **Use case**: Simple character-by-character input
- **UI**: No input field, captures keystrokes directly

### 3. CANONICAL Mode (CANON)
- **Input**: Line buffering, collected until Enter is pressed
- **Echo**: No automatic echo (depends on ECHO flag)
- **Use case**: Line-based input without echo
- **UI**: Uses input field for line editing

### 4. CANONICAL + ECHO Mode (CANON+ECHO)
- **Input**: Line buffering with Enter to send
- **Echo**: PTY echoes the complete line when received
- **Use case**: Normal shell interaction (recommended default)
- **UI**: Uses input field for line editing

## Testing the Terminal

### Expected Behavior by Mode

**CANON+ECHO (Default Shell Mode)**:
1. Type in input field: `echo "Hello"`
2. Press Enter
3. Should see: `echo "Hello"` (echoed by PTY) + `Hello` (command output)

**RAW Mode**:
1. Focus on terminal area (click the placeholder text)
2. Type characters - each keystroke sent immediately
3. PTY handles all echoing and processing

### Common Issues and Solutions

1. **Double Echo**: If you see characters twice, check:
   - Browser input field is echoing AND PTY is echoing
   - Solution: Use RAW mode or ensure input field is disabled in echo modes

2. **No Response**: If commands don't execute:
   - Check WASM module is loaded: `state.module.UTF8ToString` exists
   - Check PTY functions are exported and working
   - Verify mode is set correctly

3. **Input Not Sent**: If typing doesn't work:
   - CANON modes: Must press Enter to send line
   - RAW/ECHO modes: Must click on terminal area to focus

# ✅ Filesystem Integration Complete!

## Summary of Integration

Successfully integrated full POSIX filesystem operations into the WASM shell commands:

### Implemented Commands

1. **`ls [directory]`** - Lists directory contents with [DIR] and [FILE] indicators
2. **`cd [directory]`** - Changes working directory with validation  
3. **`pwd`** - Shows current working directory
4. **`mkdir <directory>`** - Creates directories
5. **`rmdir <directory>`** - Removes empty directories
6. **`rm <file>`** - Removes files
7. **`cat <file>`** - Displays file contents
8. **`cp <source> <dest>`** - Copies files
9. **`mv <source> <dest>`** - Moves/renames files
10. **`whoami`** - Shows current user

### Initial Directory Structure

The shell now creates a complete filesystem structure on initialization:

```
/
├── home/
│   └── user/
│       ├── README.txt (sample file with instructions)
│       ├── documents/
│       └── downloads/
├── bin/
├── usr/
│   └── bin/
├── etc/
│   └── hosts (sample system file)
├── var/
└── tmp/
```

### Testing the Filesystem

1. **Refresh the browser** to load the new WASM build
2. **Open WASM Kernel Demo** 
3. **Try these commands**:
   - `ls` - should show documents/ downloads/ README.txt
   - `cat README.txt` - should display welcome message
   - `mkdir test` - creates new directory
   - `cd test` - changes to test directory
   - `pwd` - shows /home/user/test
   - `cd ..` - goes back to parent
   - `ls` - should now show test/ directory

### Technical Implementation

- **POSIX Integration**: Used standard `opendir()`, `readdir()`, `stat()`, `mkdir()`, `fopen()`, etc.
- **Error Handling**: Proper errno handling with `strerror()` for user-friendly error messages
- **Working Directory**: Maintains current directory state with `chdir()` and `getcwd()`
- **File Operations**: Full read/write capabilities using standard C file I/O
- **Memory Management**: Safe string handling with bounds checking